## Sun B1 dynamics: nastier than expected...
When we have 3BP dynamics in E-M rotating frame, we naturally forget about $\omega$ as we normalize t* so that $\omega$ = 1 (i.e., $t_1^*$ = 2$\pi$ / (1 siderial period = 27.5days). 
This means omega should change if we want to discuss the dynamics in Sun-B1 frame ($t_2^*$ = 2$\pi$ / 365 days), while we want to stick with $t_1^*$. Therefore, the angular velocity term in the centrifugal and Coriolis force emerges: $\omega_b = 2\pi$ / (365 days / 27.5 days).

Additionally, according to Boudad's PhD thesis (C.19, P.235), the angular velocity of the E-M line should be $\omega_M = |\omega_S|/(1-|\omega_S|) ~= 1.05$, rather than $\omega_S = - t_{siderial}/t_{synodic} ~= -0.93$, which is the angular velocity of the Sun w.r.t. Earth-Moon rotating frame in BCR4P.




### normalization
$t^* = t_{syderial} = 27.5 * 86400$ [s], 

$l^*$ = Earth-Moon distance,

$m^* = m_E + m_M$.

In [1]:
using Plots
using LinearAlgebra
using DifferentialEquations
plotly()

┌ Warning: For saving to png with the `Plotly` backend `PlotlyBase` and `PlotlyKaleido` need to be installed.
│   err = ArgumentError("Package PlotlyBase not found in current path:\n- Run `import Pkg; Pkg.add(\"PlotlyBase\")` to install the PlotlyBase package.\n")
└ @ Plots C:\Users\yujit\.julia\packages\Plots\gzYVM\src\backends.jl:420


Plots.PlotlyBackend()

In [2]:
include("../src/SailorMoon.jl")   # relative path to main file of module
include("../../julia-r3bp/R3BP/src/R3BP.jl")
param3b = SailorMoon.dyanmics_parameters()


Main.SailorMoon.dynamics_params(0.987849414390376, 0.01215058560962404, 328900.5598102475, 384748.32292972936, 375700.3437894195, 388.8212386592645, -0.9251999994040079, 12.36898385069774, 0.01709689063726318, 7.601281331451572)

In [87]:
"""
Right-hand side expression for state-vector in BCR4BP
# Arguments
    - `du`: cache array of duative of state-vector, mutated
    - `u`: state-vector
    - `p`: parameters, where p = [μ2, μS, as, θ0, ωM, τ, γ, β, mdot, tmax]
        m* : mE + mL (6.0455 * 10^22)
        μ2 : mL / m* (0.012150585609624)
        μS : mS / m* (0.00000303951)
        as : (sun-B1 distance) / l* (388.709677419)
            l* : Earth-moon distance (384,400 km)
        ωM : Earth-Moon line's angular velocity around E-M barycenter
        τ  : thrust magnitude (0~1)
        γ  : thrust angle 1
        β  : thrust angle 2
        mdot : mass-flow rate
        tmax : max thrust
    - `t`: time
"""
function rhs_bcr4bp_sb1frame!(du,u,p,t)
    # unpack state
    x, y, z = u[1], u[2], u[3]
    vx, vy, vz = u[4], u[5], u[6]
    μ2, μS, as, θ0, ωM, ωb = p[1], p[2], p[3], p[4], p[5], p[6]
    τ, γ, β, mdot, tmax = p[7], p[8], p[9], p[10], p[11]

    θ = θ0 + ωM * t  # moon angle

    # create Thrust term
#     T = dv_inertial_angles([τ,γ,β])
    T = [0.0, 0.0, 0.0]
    Tx, Ty, Tz = T * tmax / u[7]  # mdot


    # derivatives of positions
    du[1] = u[4]
    du[2] = u[5]
    du[3] = u[6]

    # earth and moon location
    xe = - μ2 *cos(θ) 
    ye = - μ2 *sin(θ)
    ze = 0

    xm = (1-μ2) *cos(θ) 
    ym = (1-μ2) *sin(θ)
    zm = 0
    
#     println(xe, " ", ye, " ", ze)

    # compute distances
    r30 = sqrt((as + x)^2 + y^2 + z^2)      # SC-Sun
    r31 = sqrt((xe - x)^2 + (ye - y)^2 + (ze-z)^2)  # SC-earth
    r32 = sqrt((xm - x)^2 + (ym - y)^2 + (zm-z)^2)  # SC-Moon
#     println(r30, " ", r31, " ", r32)

    Fx = -(μS)*(x-as)/r30^3 - (1-μ2)*(x-xe)/r31^3 - μ2*(x-xm)/r32^3 + Tx
    Fy = -(μS)* y / r30^3 - (1-μ2)*(y-ye)/r31^3 - μ2*(y-ym)/r32^3 + Ty
    Fz = -(μS)* z / r30^3 - (1-μ2)*(z-ze)/r31^3 - μ2*(z-zm)/r32^3 + Tz
    # println(-(μS/(μS+1))*(x-1/(μS+1))/r30^3 , " ", - (1-μ2)/(μS+1)*(x-xe)/r31^3, " ",  - μ2/(μS+1)*(x-xm)/r32^3)
    println(Fx, " ", Fy, " ", Fz)


    du[4] =  2*ωb*vy + ωb^2*x + Fx
    du[5] = -2*ωb*vx + ωb^2*y + Fy
    du[6] =                   + Fz

    du[7] = -mdot * τ
end

rhs_bcr4bp_sb1frame!

In [88]:
"""
    transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)

Transform state from Earth-Moon rotating frame to Sun-B1 rotating frame.
Careful with sign of ωs!! (should be negative)
"""
function transform_EMrot_to_SunB1(state::Vector, θs::Real, ωs::Real)
    ωm = -ωs
    θm = π - θs
    cos_θm = cos(θm)
    sin_θm = sin(θm)
    C = [
        cos_θm      -sin_θm   0 0       0      0
        sin_θm      cos_θm    0 0       0      0
        0           0         1 0       0      0
        -ωm*sin_θm -ωm*cos_θm 0 cos_θm -sin_θm 0 
         ωm*cos_θm -ωm*sin_θm 0 sin_θm  cos_θm 0
         0          0         0 0       0      1
    ]
    state_conv = C * state    
    return state_conv
end


transform_EMrot_to_SunB1

In [89]:
function plot_circle(radius, x, y, n=50)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

plot_circle (generic function with 2 methods)

In [90]:
θM = 190 / 180 *pi 
ϕ0 = 0.0
θf = 0.0
one_year = 365.242189 * 86400
ωb = 2*pi / (one_year / param3b.tstar)

param = [param3b.mu2, param3b.mus, param3b.as, θM, param3b.oml, ωb, 0.0, 0.0, 0.0, 0.0, 0.0]

11-element Vector{Float64}:
      0.01215058560962404
 328900.5598102475
    388.8212386592645
      3.3161255787892263
     12.36898385069774
      0.07480433756791442
      0.0
      0.0
      0.0
      0.0
      0.0

In [91]:
# initial condition: Halo orbit progapation
lps = SailorMoon.lagrange_points(param3b.mu2)

## set up of initial condition (Lyapunov orbit)
lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1) # monodromy eigenvector

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12
);

# xf in the Earth-moon rotating frame
xf_em = SailorMoon.set_terminal_state2(ϕ0, θf, param3b, LPOArrival)


Halo guess Az_km: 1200.0
Linear stability ν = 618.7618470919092


6-element Vector{Float64}:
  1.1226882532375977
  2.7938137461526135e-7
 -5.228883154550508e-26
 -7.710131455619899e-7
  0.16509880733691906
  1.3395041586794773e-31

In [92]:
# change coordination from EM rot. frame to Sun-B1 frame
θs = π - θM
ωs = param3b.oms
xf_sb1 = vcat(transform_EMrot_to_SunB1(xf_em, θs, ωs), 1.0)
println(xf_sb1)

# propagate ODE
tspan = [0.0, -29]
prob = ODEProblem(rhs_bcr4bp_sb1frame!, xf_sb1, tspan, param)
sol = solve(prob, Tsit5(), reltol=1e-12, abstol=1e-12)

pcart = plot(size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4)
plot!(pcart, Array(sol)[1,:], Array(sol)[2,:], color=:blue, linewidth=1.0, label="sol", linestyle=:solid)

moon = plot_circle(1-param3b.mu2, 0.0 , 0.0)
earth = plot_circle(param3b.mu2, 0.0, 0.0)

plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")

[-1.1056320474900527, -0.19495304439972194, -5.228883154550508e-26, 0.20904042289119534, -1.1855212212722877, 1.3395041586794773e-31, 1.0]
3.613965848797184 0.2503460786944923 2.947934836516129e-25
3.613965848797184 0.2503460786944923 2.947934836516129e-25
3.5232117988675404 0.3860328885529127 2.6800868149328404e-25
3.612340970569038 0.25852905017898187 2.9472026173448413e-25
3.6103000186389975 0.26688638585264207 2.944926157408732e-25
3.6004634930559263 0.2949026755068822 2.9252886415099484e-25
3.598755591233849 0.29869052371686206 2.921118552658545e-25
3.598316912734243 0.2996305323898041 2.920024224342221e-25
3.5983170286051145 0.2996306402035058 2.9200248776147536e-25
3.6129465046647358 0.2556889372753973 2.9476235243120607e-25
3.6117395661929397 0.26116600098198256 2.94665522366213e-25
3.6063694425666797 0.2797649911712902 2.9382685807133304e-25
3.605473208017529 0.28231827610334714 2.936479765131597e-25
3.6052440025687784 0.2829536909660776 2.936009878208313e-25
3.605244058554371

3.139141689719803 0.3644931691995967 1.1426762010518716e-25
3.139141703319391 0.36449318758523414 1.1426762512165993e-25
3.136602736123461 0.36276090022336127 1.131593629911386e-25
3.13402809005266 0.3609763352469182 1.1203414962313764e-25
3.1254685086536185 0.35483501675542817 1.0828310892707762e-25
3.1243125475497555 0.35398043955321257 1.077753125332111e-25
3.1240254975002806 0.3537672799779145 1.0764916990342814e-25
3.124025511140707 0.35376729809127816 1.076491746489975e-25
3.1216442501331634 0.35198429132912334 1.0660203302145436e-25
3.1192316671168356 0.3501508095369824 1.0553982013388795e-25
3.111226379650612 0.3438657456598467 1.0200567232575703e-25
3.1101471074266342 0.34299405223622953 1.0152804979914729e-25
3.109879168627696 0.3427767292412246 1.0140943272587832e-25
3.10987918223506 0.3427767468464557 1.0140943716362862e-25
3.1076699909915613 0.34097083257640154 1.0043076510721446e-25
3.1054333966473227 0.33911671632137863 9.943874661720827e-26
3.098023958402736 0.332782352

2.982201417608966 0.14146156510124963 4.06952347858669e-26
2.9820865829852403 0.14069247327764428 4.061101187760863e-26
2.9817234109967528 0.13810581448686995 4.0335704646127224e-26
2.9816766510404884 0.13775267958823997 4.0299072811691806e-26
2.9816651270215324 0.13766483838968308 4.02899963989869e-26
2.9816651696923757 0.1376648473535782 4.0289999294993437e-26
2.981570414300032 0.13693073030098946 4.0214693822382514e-26
2.9814771805144322 0.13618256217496946 4.013897917780393e-26
2.981185875672759 0.13366494232466147 3.989171902029564e-26
2.981148839495936 0.1333210660339682 3.985884816045393e-26
2.981139731154176 0.13323552143669998 3.9850704737556e-26
2.981139777652462 0.13323553074022146 3.985070786104205e-26
2.981065108564212 0.1325200963967332 3.9783120850220885e-26
2.980992288096731 0.13179078643930348 3.971520293775304e-26
2.980769000864505 0.12933515753790836 3.949363968674534e-26
2.9807411809963913 0.12899956590945963 3.9464214231060074e-26
2.980734362799606 0.12891607521997

3.033636021557689 -0.024673657015983147 3.9368292448867615e-26
3.0345443942034724 -0.02614208325164814 3.9423903320282106e-26
3.0376795822111413 -0.031218457851554058 3.9616944222967854e-26
3.0381192101500187 -0.03192858309633313 3.964415017873273e-26
3.0382290584988314 -0.0321058848907291 3.9650953405740463e-26
3.038230552950596 -0.03210605729917053 3.9651054104075275e-26
3.0391691853209015 -0.033627530056080795 3.9709261773961904e-26
3.0401511424390177 -0.03519871236435207 3.9770343383953423e-26
3.043535174966828 -0.040635180871935066 3.998198244079879e-26
3.0440091846641946 -0.04139632656740142 4.0011767858544077e-26
3.0441276070779897 -0.041586393535764626 4.001921470860301e-26
3.0441293011426755 -0.04158662252223663 4.0019329662992235e-26
3.0451250649457053 -0.043192859355301906 4.008202669336758e-26
3.0461667781453055 -0.04485256985252348 4.014779714921351e-26
3.0497507437724862 -0.05060078667552311 4.0375241264946135e-26
3.050252130199134 -0.05140630273036502 4.0407202902355135e

3.1734415466319605 -0.38204561359430345 5.1000875496737006e-26
3.173442421478095 -0.38204625892902905 5.100094609812195e-26
3.173751068544841 -0.38422253803777295 5.10646575696006e-26
3.174071018769878 -0.38648348024659535 5.113143991137481e-26
3.1751314455734807 -0.394392222254457 5.1366758080908305e-26
3.1752749918699115 -0.395510644762514 5.1400336603241864e-26
3.1753106680440077 -0.39579036821665914 5.140874881603942e-26
3.1753114445994663 -0.3957909562357125 5.140881201419619e-26
3.1755771860541104 -0.3979148262697788 5.1472741079467415e-26
3.1758519328443655 -0.4001211771077914 5.153976816350282e-26
3.1767550924310366 -0.407840190665899 5.177659696021629e-26
3.1768763630667545 -0.4089319272005261 5.181046635844039e-26
3.1769064630238097 -0.4092049815205312 5.181895420676235e-26
3.1769071502225055 -0.4092055145727019 5.181901060086316e-26
3.177130486642388 -0.4112765366767898 5.188349798690521e-26
3.177360558132655 -0.4134279343402465 5.195113803231878e-26
3.1781076949337863 -0.42

...

0.6164808466117138 -0.2822753161235226 -1.824831063776201e-27
0.6163637706222448 -0.28233877750737946 -1.824834907218056e-27
0.6163637584996796 -0.28233905031779066 -1.8248345298615848e-27
0.6154205777917116 -0.28284992057717784 -1.8248632835045916e-27
0.614454859049736 -0.2833784566484183 -1.8248956750802238e-27
0.6111462931400553 -0.2851997643134672 -1.8249981913947877e-27
0.6106879383357315 -0.28545407708900056 -1.8250122247112563e-27
0.6105736783079341 -0.2855175615484847 -1.8250157123126897e-27
0.6105736781879323 -0.2855178349769936 -1.8250153554762062e-27
0.6096533958211477 -0.2860287713224176 -1.825041257607072e-27
0.6087112590381181 -0.28655737366640177 -1.825070873764605e-27
0.6054837125719926 -0.28837893306431034 -1.825164347571289e-27
0.6050366266634214 -0.2886332828654077 -1.825177195378089e-27
0.6049251779336072 -0.2886967766473819 -1.8251803906665927e-27
0.6049251896294624 -0.28869705056633665 -1.8251800545295642e-27
0.6040280479756125 -0.28920777719604607 -1.825203611679

0.509648213457503 -0.3633070185560217 -1.8310036968758584e-27
0.509648394720963 -0.36330720724729737 -1.8310038419043896e-27
0.5093003148156668 -0.36372107042003504 -1.831072999500777e-27
0.5089452803583793 -0.3641489639422808 -1.831148102687907e-27
0.5077305483833259 -0.3656245865785258 -1.8314073882809916e-27
0.5075626313760712 -0.36583070247995514 -1.8314442730321734e-27
0.5075207927476048 -0.3658821566932564 -1.8314535137094873e-27
0.5075209727264117 -0.365882338232326 -1.8314536674895976e-27
0.5071910345300435 -0.36628785147831866 -1.831525511175673e-27
0.5068545320884216 -0.3667070889753323 -1.8316032710994904e-27
0.5057032007795035 -0.36815291314503457 -1.8318718381608782e-27
0.5055440505549647 -0.36835486880552365 -1.8319100065536615e-27
0.5055043966124548 -0.368405284409561 -1.8319195671848873e-27
0.5055045745182245 -0.3684054584678361 -1.8319197283690274e-27
0.5051925790116801 -0.368801885372758 -1.831993980264518e-27
0.5048743954133516 -0.36921170445725227 -1.832074095253811

0.4744960028859863 -0.43525217695947127 -1.8552117323762414e-27
0.4744962995839474 -0.4352524249417819 -1.85521210801432e-27
0.47441232163063796 -0.43574708839363463 -1.855470437704399e-27
0.4743304701183407 -0.43625879251720046 -1.8557432433073974e-27
0.4740588171443422 -0.43802419779981866 -1.856688467133149e-27
0.4740227505794716 -0.4382709066446302 -1.8568219256885813e-27
0.4740138315654995 -0.4383324999448961 -1.8568553095851335e-27
0.47401413509941354 -0.4383327524492261 -1.85685569743004e-27
0.47394165886044365 -0.43883224788653175 -1.857125269119972e-27
0.47387172734700617 -0.43934896566423076 -1.857409813175626e-27
0.4736412877694083 -0.4411317048820233 -1.858395724567785e-27
0.47361099328668016 -0.44138084197770305 -1.858534906236924e-27
0.47360351585329225 -0.44144304180688276 -1.8585697206565952e-27
0.4736038263029482 -0.44144329883932054 -1.858570120976662e-27
0.4735430613530334 -0.4419475731302001 -1.858851206667544e-27
0.473485264472438 -0.44246924954611533 -1.8591477687

0.5399454031267574 -0.6058193407350262 -2.0275844250059155e-27
0.5421149719979511 -0.6084285281267376 -2.031436983609378e-27
0.5424206838647808 -0.6087935776758923 -2.0319785338954818e-27
0.5424971178797358 -0.6088847349745216 -2.032113883838521e-27
0.5424978078530102 -0.6088852009586623 -2.0321150704688498e-27
0.5431151017244076 -0.609621108468732 -2.0332066056151933e-27
0.5437590198960236 -0.6103830973865064 -2.034345431693818e-27
0.5459972627016589 -0.6130153029714731 -2.0382905150111504e-27
0.5463125967045531 -0.6133835822220275 -2.038845044517864e-27
0.5463914339574938 -0.6134755464069335 -2.038983637045433e-27
0.5463921349696217 -0.6134760179193355 -2.0389848473059437e-27
0.5470287729092718 -0.6142183561462381 -2.0401024431130317e-27
0.5476927088755582 -0.6149870134437966 -2.041268303242105e-27
0.5500002591529831 -0.6176423326439529 -2.0453070397107006e-27
0.5503253043210752 -0.6180138550130303 -2.0458747064053325e-27
0.5504065670556789 -0.6181066294357844 -2.046016580962783e-27


0.6823734359528493 -0.7390043517589694 -2.2630915050000208e-27
0.6823738848534212 -0.7390046169076023 -2.2630923779764798e-27
0.683191435208335 -0.7396407975808086 -2.264389680939591e-27
0.684038978242528 -0.7402987525870371 -2.265735760693156e-27
0.6869756846365105 -0.7425722003446805 -2.270394189822914e-27
0.6873876646707621 -0.7428902396169268 -2.271047311618903e-27
0.687490581559469 -0.7429696519687549 -2.2712104598155628e-27
0.6874910464992166 -0.7429699262960883 -2.2712113633154333e-27
0.6883400662719981 -0.7436247654383179 -2.272556349480054e-27
0.6892203060561329 -0.7443020664957335 -2.273952052400223e-27
0.6922705438754236 -0.7466424466494679 -2.2787825324524864e-27
0.6926984855710541 -0.7469698647165587 -2.2794598302570543e-27
0.6928053914680183 -0.7470516197335039 -2.279629020213784e-27
0.692805883544307 -0.7470519099499712 -2.2796299746100935e-27
0.693683346534926 -0.7477226469536103 -2.2810176985923693e-27
0.6945931257850113 -0.7484164311816621 -2.282457856715378e-27
0.697